### Map results back to shapefile
The functions below prepare the data for the process of mapping the results back to their original shapefiles by implementing a "table join" between the outputs .csv and the attribute table of the shapefile

In [ ]:
import pandas as pd
import os, sys
sys.path.append(r'C:\Users\gost_\Desktop\lima\GOSTNets\GOSTNets')
import GOSTnet as gn
import importlib
importlib.reload(gn)
import geopandas as gpd
import rasterio
from rasterio import features
from shapely.wkt import loads
import numpy as np

In [ ]:
#define the paths to the origins, destinations and .pickle file
pth = r'C:\Users\gost_\Desktop\lima\data\OD'
pckle = r'biggest_subg.pickle'
walk_speed = 4 
WGS = {'init':'epsg:4326'}
measure_crs = {'init':'epsg:32718'}

In [ ]:
# read  file
OD_name = r'output_gt_origins_destinations.csv'
OD = pd.read_csv(os.path.join(pth, OD_name), sep=';')
#OD = OD.rename(columns = {'Unnamed: 0':'O_ID'})
OD_ = OD.set_index('O_ID')
OD = OD_.replace([np.inf, -np.inf], np.nan)
OD['Minimum'] = OD.loc[:,:].min(axis=1)

In [ ]:
OD[:10]

In [ ]:
### PREPARE ORIGIN FILE 
# set index to nearest node to match on the travel time from that node to nearest destination (OD['Minimum'])
orig = pd.read_csv(os.path.join(pth, 'origins_snapped.csv'))
orig = orig.rename({'ID':'OBJECTID'}, axis = 1) # rename ID column to 'OBJECTID'
orig = orig.set_index('NN') # used for matching on the OD matrix
orig[:10]

In [ ]:
orig['closest_hosp'] = OD['Minimum'] # here we match on closest hospital 
orig[:5]

In [ ]:
# add on walktime for time computation
walkspeed = 4
orig['walktime'] = orig['NN_dist'] / 1000 / walkspeed * 3600
orig['total_time'] = (orig['walktime'] + orig['closest_hosp']) / 60
###   OR   ###
#compute total distance: NN_dist + dist
orig['total_distance'] = (orig['NN_dist'] + orig['closest_hosp'])

In [ ]:
#read in the origins in shapefile format
fp = r'C:\Users\gost_\Desktop\lima'
shpfil = gpd.read_file(os.path.join(fp, 'VillaElSalvador_urban.shp'))

In [ ]:
# Ensure both shapefile AND origin file have a non-duplicate index to enable matching of results on to the shapefile
shpfil = shpfil.set_index('OBJECTID')
orig = orig.set_index('OBJECTID')

In [ ]:
# Prep output shapefile, match on visualization column, save file
shpfil['total_time'] = orig['total_time']
shpfil = shpfil.to_crs({'init':'epsg:4326'})
shpfil.to_file(os.path.join(pth, 'lima_origins_w_time.shp'), driver = 'ESRI Shapefile')

In [ ]:
# write the road file as a shapefile
from shapely.wkt import loads
edges = pd.read_csv(os.path.join(pth, 'biggest_subg_edges.csv'))
edges['geometry'] = edges['geometry'].apply(loads)
edges_gdf = gpd.GeoDataFrame(edges, geometry = 'geometry', crs = {'init':'epsg:4326'})
edges_gdf.to_file(os.path.join(fp, 'VES_roads.shp'), driver = 'ESRI Shapefile')